<a href="https://colab.research.google.com/github/aashu1328/Natural-language-Processing/blob/master/NLP_M2_J058.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Q1

### IMPORTING LIBRARIES

In [1]:
import pandas as pd
import csv
import nltk 
import sklearn
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import LatentDirichletAllocation as LDA
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
! pip install cltk
! pip install pyLDAvis
import re
import string
from cltk.tokenize.sentence import TokenizeSentence

In [0]:
#df=pd.read_csv("/content/train_telugu_news.csv")
#df.head() 

### TRAIN DATASET

In [0]:
#reading through  github
url = "https://media.githubusercontent.com/media/reubence/NLP-COURSEWORK/master/Datasets/telgu/train_telugu_news.csv"
df_art = pd.read_csv(url, encoding = "utf-8’")

In [5]:
#engine='python',quoting=csv.QUOTE_NONE
#encoding='utf-8'
#encoding= 'unicode_escape'
df_art.head()

,SNo,date,heading,body,topic
0,414,11-05-2017 00:39:13,ఐడిబిఐపై ఆర్‌బిఐ నజర్‌,భారీ ఎత్తున మొండిబకాయిలు పెరిగిపోవడంతో ఐడిబిఐ ...,business
1,2668,12-06-2017 00:40:32,బ్యాంకింగ్‌ చీఫ్‌లతో నేడు జైట్లీ భేటీ,న్యూఢిల్లీ : ఆర్థిక మంత్రి అరుణ్‌ జైట్లీ సోమవా...,business
2,19923,19-01-2017 19:51:31,కీలక వికెట్ తీసిన జడేజా..,కటక్: ఇంగ్లండ్‌తో జరుగుతున్న సెకండ్ వన్డే మ్యా...,sports
3,15762,23-11-2017 17:29:04,మరో రెచ్చగొట్టే చర్యకు దిగిన పాకిస్థాన్,\nఇస్లామాబాద్ : పాకిస్థాన్ అంతర్జాతీయ ఉగ్రవాది...,nation
4,8624,03-04-2017 15:48:23,గోవాలో కొడుకుతో కలిసి అల్లు అర్జున్ స్విమ్మింగ్!,స్టార్‌ హీరోగా వరుస సినిమాలతో బిజీగా ఉన్నప్పటి...,entertainment


In [6]:
#df1 = df.iloc[:,2:5]
df_art.columns
df_art.shape

(17312, 5)

In [7]:
df1=df_art[['heading','body','topic']]
df1.head()

,heading,body,topic
0,ఐడిబిఐపై ఆర్‌బిఐ నజర్‌,భారీ ఎత్తున మొండిబకాయిలు పెరిగిపోవడంతో ఐడిబిఐ ...,business
1,బ్యాంకింగ్‌ చీఫ్‌లతో నేడు జైట్లీ భేటీ,న్యూఢిల్లీ : ఆర్థిక మంత్రి అరుణ్‌ జైట్లీ సోమవా...,business
2,కీలక వికెట్ తీసిన జడేజా..,కటక్: ఇంగ్లండ్‌తో జరుగుతున్న సెకండ్ వన్డే మ్యా...,sports
3,మరో రెచ్చగొట్టే చర్యకు దిగిన పాకిస్థాన్,\nఇస్లామాబాద్ : పాకిస్థాన్ అంతర్జాతీయ ఉగ్రవాది...,nation
4,గోవాలో కొడుకుతో కలిసి అల్లు అర్జున్ స్విమ్మింగ్!,స్టార్‌ హీరోగా వరుస సినిమాలతో బిజీగా ఉన్నప్పటి...,entertainment


In [8]:
#df1['merge']=df['heading']+df['body']
#stop_words = stopwords.words('telugu')
print (stopwords.fileids())

['arabic', 'azerbaijani', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [0]:
file = open('/content/telugu') # Acessing a telegu stopword file from outside 
text = file.read()
stop_words=text.split()
#t

In [0]:
def remove_stopwords(a):
    a=[word for word in a.split() if word not in stop_words]
    return ''.join(a)

In [11]:
df1["stop"]=df1.body.apply(remove_stopwords)
df1.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,heading,body,topic,stop
0,ఐడిబిఐపై ఆర్‌బిఐ నజర్‌,భారీ ఎత్తున మొండిబకాయిలు పెరిగిపోవడంతో ఐడిబిఐ ...,business,భారీఎత్తునమొండిబకాయిలుపెరిగిపోవడంతోఐడిబిఐవ్యవహ...
1,బ్యాంకింగ్‌ చీఫ్‌లతో నేడు జైట్లీ భేటీ,న్యూఢిల్లీ : ఆర్థిక మంత్రి అరుణ్‌ జైట్లీ సోమవా...,business,న్యూఢిల్లీ:ఆర్థికమంత్రిఅరుణ్‌జైట్లీసోమవారంప్రభ...
2,కీలక వికెట్ తీసిన జడేజా..,కటక్: ఇంగ్లండ్‌తో జరుగుతున్న సెకండ్ వన్డే మ్యా...,sports,కటక్:ఇంగ్లండ్‌తోజరుగుతున్నసెకండ్వన్డేమ్యాచ్‌లో...
3,మరో రెచ్చగొట్టే చర్యకు దిగిన పాకిస్థాన్,\nఇస్లామాబాద్ : పాకిస్థాన్ అంతర్జాతీయ ఉగ్రవాది...,nation,ఇస్లామాబాద్:పాకిస్థాన్అంతర్జాతీయఉగ్రవాదిహపీజ్స...
4,గోవాలో కొడుకుతో కలిసి అల్లు అర్జున్ స్విమ్మింగ్!,స్టార్‌ హీరోగా వరుస సినిమాలతో బిజీగా ఉన్నప్పటి...,entertainment,స్టార్‌హీరోగావరుససినిమాలతోబిజీగాఉన్నప్పటికీకుట...


In [0]:
def clean_word(temp):
    temp = temp.translate(str.maketrans('', '', string.punctuation))
    temp = temp.replace("\u200c","")
    temp = re.sub(r'[A-Za-z0-9]', ' ', temp)
    temp = re.sub(r'[0-9]', ' ', temp)
    tokenizer = TokenizeSentence('telugu')
    word_tokens = tokenizer.tokenize(temp)
    t = [temp.lower() for w in word_tokens if not temp.lower() in stop_words]
    return ''.join(t)  

In [13]:
df1["stop"]=df1.stop.apply(clean_word)
df1.stop[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'భారీఎత్తునమొండిబకాయిలుపెరిగిపోవడంతోఐడిబిఐవ్యవహారాలపైభారతరిజర్వుబ్యాంకుఆర్బిఐకన్నేసిందిఐడిబిఐబ్యాంకునుఆర్బిఐవాచ్లి్స్టలోచేర్చినట్టుగాతెలిసిందినికరమొండిపద్దులమొత్తం శాతందాటడంవరసగారెండేళ్లపాటునష్టాలనుప్రకటించడంకాపిటల్అడెక్వసీనిర్దేశితప్రమాణాలకంటేతగ్గడంఈసందర్భాల్లోబ్యాంకులనుఆర్బిఐవాచ్లిస్ట్లోచేరుస్తుందితమబ్యాంకుకుసంబంధించిఆర్బిఐప్రాంప్ట్కరెక్టీవ్యాక్షన్పిసిఎచేపట్టినట్టుఐడిబిఐవెల్లడించిందిగతంలోఓవర్సీస్బ్యాంక్యునైటెడ్బ్యాంక్కుసంబంధించిఆర్బిఐపిసిఎచేపట్టిందిపిసిఎవల్లబ్యాంకుపైఆర్థికంగాభారంపడదనీదీనివల్లఅంతర్గతనియంత్రణలుమెరుగుపడటంతోపాటుకార్యకలాపాలుమెరుగవుతాయనిఐడిబిఐవివరించిందిఆర్బిఐఆదేశాలకుఇప్పుడుమూలధనాన్నిపొదుపుగావినియోగించడంపైఐడిబిఐదృష్టిసారించాల్సిఉంటుందిఇందులోభాగంగాపరపతిపైనియంత్రణలువిధించడంనియామకాలనునిలిపేయడంకొత్తపెట్టుబడిపథకాలనుపక్కనబెట్టడంవంటిచర్యలుతీసుకోవాల్సిఉంటుందిబ్యాంకుసరైనరీతిలోస్పందించనిపక్షంలోమరోబలమైనబ్యాంకులోవిలీనానికిసిద్ధంకావాల్సిఉంటుంది'

In [0]:
df1=df1.drop('body',1)

In [15]:
model=TfidfVectorizer()
core=model.fit_transform(df1['stop'])
final=pd.DataFrame(core.toarray(),columns=model.get_feature_names())
final.head()

,üక,అఆ,అఆఇఈఅ,అఆక,అఆత,అఆప,అఆశతమ,అఆస,అఇమ,అక,అకట,అకడ,అకడమ,అకర,అకవ,అకస,అఖ,అఖర,అఖల,అగ,అగచ,అగట,అగడ,అగడగ,అగత,అగన,అగప,అగమ,అగర,అగస,అఘ,అఙ,అచ,అచటప,అచల,అఛ,అజ,అజన,అజనబ,అజమ,...,హహక,హహస,హహహక,ఽఉల,ఽక,ఽఛ,ఽడ,ఽఢత,ఽథ,ఽథన,ఽథప,ఽథమ,ఽథర,ఽద,ఽదప,ఽదవ,ఽధ,ఽధకచట,ఽధన,ఽధబ,ఽధమ,ఽధర,ఽధరన,ఽధరలయ,ఽధవ,ఽధవనశతకదర,ఽప,ఽబ,ఽభ,ఽమ,ఽళదర,ఽవ,ఽవస,ఽశ,ఽస,౨జ,౩క,౩౦ఆద,౩౩ప,౪శ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
fizz=pd.get_dummies(df1.topic)

In [17]:
rfc=RandomForestClassifier(random_state=42)
#param_grid={'n_estimators': [200, 500],'max_features': ['auto', 'sqrt', 'log2'],'max_depth' : [4,5,6,7,8],'criterion' :['gini', 'entropy']}
#CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
rfc.fit(final,fizz)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

### TEST DATASET

In [0]:
url = "https://media.githubusercontent.com/media/reubence/NLP-COURSEWORK/master/Datasets/telgu/test_telugu_news.csv"
df_test = pd.read_csv(url, encoding = "utf-8’")

In [19]:
df_test.columns
df_test.shape

(4329, 5)

In [20]:
df2=df_test[['heading','body','topic']]
df2.head()

,heading,body,topic
0,కేశవ,హీరోగా తెలుగు సినిమాల్లోకి ఎంట్రీ ఇచ్చిన నిఖిల...,entertainment
1,డబ్బుల కోసం ప్రాణాల మీదకు తెచ్చుకుంటారా.. హీరో...,సినిమాల్లో యాక్షన్ స్టంట్లు చేసేటప్పుడు ఎక్కు...,entertainment
2,దక్షిణాదిన బీజేపీ పప్పులుడకవ్‌,దక్షిణ భారతాన్ని ఆక్రమించేందుకు బీజేపీ పంచెలు ...,nation
3,పీఎస్‌ఎల్వీ-సీ39కి గ్రీన్‌సిగ్నల్‌,నేటి మధ్యాహ్నం 1.59కి కౌంట్‌డౌన్‌ షురూశ్రీహరి...,nation
4,ఏయ్ జయప్రకాష్ అనే కేక వినిపించింది.. వెనక్కి త...,‘ఏమి రా... అబ్బి... యాడికి పోయినావు’ అంటూ రాయ...,entertainment


In [21]:
df2["stop"]=df2.body.apply(remove_stopwords)
df2.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,heading,body,topic,stop
0,కేశవ,హీరోగా తెలుగు సినిమాల్లోకి ఎంట్రీ ఇచ్చిన నిఖిల...,entertainment,హీరోగాతెలుగుసినిమాల్లోకిఎంట్రీఇచ్చిననిఖిల్‌కుస...
1,డబ్బుల కోసం ప్రాణాల మీదకు తెచ్చుకుంటారా.. హీరో...,సినిమాల్లో యాక్షన్ స్టంట్లు చేసేటప్పుడు ఎక్కు...,entertainment,సినిమాల్లోయాక్షన్స్టంట్లుచేసేటప్పుడుఎక్కువగాహీ...
2,దక్షిణాదిన బీజేపీ పప్పులుడకవ్‌,దక్షిణ భారతాన్ని ఆక్రమించేందుకు బీజేపీ పంచెలు ...,nation,దక్షిణభారతాన్నిఆక్రమించేందుకుబీజేపీపంచెలుబిగిం...
3,పీఎస్‌ఎల్వీ-సీ39కి గ్రీన్‌సిగ్నల్‌,నేటి మధ్యాహ్నం 1.59కి కౌంట్‌డౌన్‌ షురూశ్రీహరి...,nation,"నేటిమధ్యాహ్నం1.59కికౌంట్‌డౌన్‌షురూశ్రీహరికోట,ఆ..."
4,ఏయ్ జయప్రకాష్ అనే కేక వినిపించింది.. వెనక్కి త...,‘ఏమి రా... అబ్బి... యాడికి పోయినావు’ అంటూ రాయ...,entertainment,‘ఏమిరా...అబ్బి...యాడికిపోయినావు’అంటూరాయలసీమయాస...


In [22]:
df2["stop"]=df2.stop.apply(clean_word)
df2.stop[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'హీరోగాతెలుగుసినిమాల్లోకిఎంట్రీఇచ్చిననిఖిల్కుస్వామిరారాసినిమాపెద్దబ్రేక్ఇచ్చిందిఆసినిమాదర్శకుడుసుధీర్వర్మనిఖిల్కుమంచిస్నేహితుడుకూడాస్వామిరారానుండినిఖిల్కార్తికేయఎక్కడికిపోతావుచిన్నవాడాచిత్రాలతోసక్సెస్ఫుల్హీరోగాపేరుతెచ్చుకున్నాడుక్రైమ్కామెడినేపథ్యంలోస్వామిరారాచిత్రాన్నిరూపొందించినసుధీర్వర్మనిఖిల్తోమరోసినిమాచేస్తున్నాడనగానేఈహిట్ద్వయంఎలాంటిసినిమాచేస్తుందోఈసారిఎలాంటిజోనర్లోమూవీచేస్తారోననిఅనుకున్నారునిఖిల్సుధీర్వర్మకాంబినేషన్లోరివేంజ్డ్రామాగాకేశవచిత్రంరూపొందుతోందనితెలియగానేవిభిన్నమైనస్క్రిప్ట్లనుఎన్నుకునేనిఖిల్కేశవలోఎలాకనపడనున్నాడోననిఅనుకున్నారుమరికేశవఅందరిఊహలనురీచ్అయ్యిందాలేదాఅనితెలుసుకుందాంసమర్పణదేవాన్ష్నామానిర్మాణసంస్థఅభిషేక్పిక్చర్స్తారాగణంనిఖిల్రీతూవర్మప్రియదర్శివెన్నెలకిషోర్నందకిషోర్మధునందన్అజయ్రావురమేష్బ్రహ్మాజీరాజారవీంద్రతదితరులుసంగీతంసన్నిఎంఆర్ఛాయాగ్రహణందివాకర్మణినేపథ్యసంగీతంప్రశాంత్పిళ్ళైకూర్పుఎస్ఆర్శేఖర్నిర్మాతఅభిషేక్నామాకథకథనందర్శకత్వంసుధీర్వర్మకథకేశవనిఖిల్కిగుండెఅందరిలాకాకుండాకుడివైపుఉండటంతోడాక్టర్స్అతనుహైఎమోషనల్ఫీలింగ్స్కులోనుకాకూడదనిచెబుతారుకాబట్టివిషయాన్నిఅయినాటెన్షన్లేకుండా

In [0]:
df2=df2.drop('body',1)

In [24]:
core=model.transform(df2['stop'])
final2=pd.DataFrame(core.toarray(),columns=model.get_feature_names())
final2.head()

,üక,అఆ,అఆఇఈఅ,అఆక,అఆత,అఆప,అఆశతమ,అఆస,అఇమ,అక,అకట,అకడ,అకడమ,అకర,అకవ,అకస,అఖ,అఖర,అఖల,అగ,అగచ,అగట,అగడ,అగడగ,అగత,అగన,అగప,అగమ,అగర,అగస,అఘ,అఙ,అచ,అచటప,అచల,అఛ,అజ,అజన,అజనబ,అజమ,...,హహక,హహస,హహహక,ఽఉల,ఽక,ఽఛ,ఽడ,ఽఢత,ఽథ,ఽథన,ఽథప,ఽథమ,ఽథర,ఽద,ఽదప,ఽదవ,ఽధ,ఽధకచట,ఽధన,ఽధబ,ఽధమ,ఽధర,ఽధరన,ఽధరలయ,ఽధవ,ఽధవనశతకదర,ఽప,ఽబ,ఽభ,ఽమ,ఽళదర,ఽవ,ఽవస,ఽశ,ఽస,౨జ,౩క,౩౦ఆద,౩౩ప,౪శ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.053067,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
fizz2=pd.get_dummies(df2.topic)

In [0]:
pred = rfc.predict(final2)

In [27]:
pred.shape
fizz2.shape

(4329, 5)

In [28]:
#conf_mat = confusion_matrix(fizz2,pred)
#print(conf_mat)
confusion_matrix(fizz2.values.argmax(axis=1), pred.argmax(axis=1))

array([[ 580,    1,    0,   72,    0],
       [ 179,   58,    0,   39,    1],
       [ 311,    0,  946,   32,    0],
       [ 419,    2,    9, 1243,    0],
       [ 266,    1,    7,   24,  139]])

In [29]:
target_names = ['business', 'editorial', 'entertainment','nation','sports']
print(sklearn.metrics.classification_report(fizz2,pred,target_names=target_names))

               precision    recall  f1-score   support

     business       0.99      0.21      0.34       653
    editorial       0.94      0.21      0.34       277
entertainment       0.98      0.73      0.84      1289
       nation       0.88      0.74      0.81      1673
       sports       0.99      0.32      0.48       437

    micro avg       0.93      0.58      0.72      4329
    macro avg       0.96      0.44      0.56      4329
 weighted avg       0.94      0.58      0.68      4329
  samples avg       0.58      0.58      0.58      4329



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Q2 TOPIC MODELLING AND LDA 

In [0]:
#reading through github
url = "https://media.githubusercontent.com/media/aashu1328/Natural-language-Processing/master/Data/telugu_books.csv"
df_topic = pd.read_csv(url, encoding = "utf-8’")

In [31]:
df_topic.head()

,SNo,text
0,0,\r\n సుశీలమ్మ కళ్ళలో భయం పారాడింది.\r\n ...
1,1,...
2,2,"""ఎలా వుంది ఫ్రెండ్..... ""\n\r\n ""టప్ టప..."
3,3,"""అది ఊరుకున్నా మీరు వుండలేరు. మీరిద్దరే ..."
4,4,అయన అలా వెళ్ళిపోగానే గుమ్మం పక్కనుంచి రాజ...


In [32]:
df_topic=df_topic.iloc[0:100,:]
df_topic.head()

,SNo,text
0,0,\r\n సుశీలమ్మ కళ్ళలో భయం పారాడింది.\r\n ...
1,1,...
2,2,"""ఎలా వుంది ఫ్రెండ్..... ""\n\r\n ""టప్ టప..."
3,3,"""అది ఊరుకున్నా మీరు వుండలేరు. మీరిద్దరే ..."
4,4,అయన అలా వెళ్ళిపోగానే గుమ్మం పక్కనుంచి రాజ...


In [33]:
df_topic.dtypes
df_topic.isna().sum()

SNo     0
text    0
dtype: int64

In [0]:
df_topic["new"]=df_topic.text.apply(clean_word)
#df_topic.new[0]

In [35]:
df_topic["new"]=df_topic.new.apply(remove_stopwords)
df_topic.head()

,SNo,text,new
0,0,\r\n సుశీలమ్మ కళ్ళలో భయం పారాడింది.\r\n ...,సుశీలమ్మకళ్ళలోభయంపారాడిందిఅనాధబిడ్డఅనిచిన్నప్ప...
1,1,...,దశరథునిమరణముపశ్చాత్తాపముతోకుమిలిపోవుచున్ననేనప్...
2,2,"""ఎలా వుంది ఫ్రెండ్..... ""\n\r\n ""టప్ టప...",ఎలావుందిఫ్రెండ్టప్టపాటప్ఎక్కణ్నుంచోవస్తున్నట్ట...
3,3,"""అది ఊరుకున్నా మీరు వుండలేరు. మీరిద్దరే ...",అదిఊరుకున్నామీరువుండలేరుమీరిద్దరేవుండండినేనువె...
4,4,అయన అలా వెళ్ళిపోగానే గుమ్మం పక్కనుంచి రాజ...,అయనఅలావెళ్ళిపోగానేగుమ్మంపక్కనుంచిరాజుబయటకువస్త...


In [0]:
df_top=df_topic.drop('text',1)

In [37]:
vect=TfidfVectorizer()
core=vect.fit_transform(df_top['new'])
f=pd.DataFrame(core.toarray(),columns=vect.get_feature_names())
f.head()

,అఆల,అక,అకల,అకస,అఖ,అగ,అగర,అఘ,అచ,అచట,అజ,అజయ,అట,అడ,అడక,అడగ,అడగక,అడగకప,అడగట,అడగడ,అడగదల,అడగబ,అడగల,అడగవచ,అడవ,అణ,అణచ,అత,అతడ,అతడక,అతడన,అతణ,అతన,అతనన,అతనల,అతనవసరమ,అతప,అద,అదన,అదల,...,హతమ,హతమగ,హతవ,హద,హన,హనక,హనన,హనమ,హనర,హనల,హప,హపడ,హబ,హమ,హర,హరధనమ,హరమ,హల,హలతఅతన,హలతఆమ,హలతఒప,హలతఓప,హలతచ,హలతమ,హలతమధ,హలతర,హవ,హవచ,హస,హసగ,హసమ,హహహనవ,౦చ,౦ట,౦డ,౦డవ,౦ద,౦నవలల,౧ఇరవ,౧న
0,0.0,0.000000,0.0,0.0,0.0,0.118665,0.0,0.0,0.0,0.0,0.0,0.0,0.034060,0.021864,0.0,0.0,0.08816,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.038779,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.038380,0.0,0.0,0.0,0.0,0.0,0.0,0.022032,0.028286,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.042513,0.0,0.0,0.026872,0.049807,0.0,0.0,0.0,0.015348,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.042513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.022999,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.037734,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.066443,0.0,0.0,0.0,0.020474,0.0,0.0,0.0,0.0,0.066927,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.027134,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.089039,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.089755,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.098702,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.042593,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.011647,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.035010,0.0,0.0,0.000000,0.102543,0.0,0.0,0.0,0.012639,0.0,0.0,0.0,0.0,0.072303,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
warnings.simplefilter("ignore", DeprecationWarning)

In [0]:
def topics(model, vect, n_top_words):
    words = vect.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [40]:
lda = LDA(n_components=6, n_jobs=-1)
lda.fit(f)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=6, n_jobs=-1,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [41]:
topics(lda,vect,12)


Topic #0:
రళయ ళనజర సహజ తరలఅన కఒకర పఅ మష షద ఇనప ఆనర భయపడక నతపనఆయనఇ

Topic #1:
రళయ షద ఇనప నతపనఆయనఇ ళవచ ఈమద భయపడక లగక ఆనర గవస టశక ళనజర

Topic #2:
రళయ షద ఇనప నతపనఆయనఇ ళవచ ఈమద భయపడక లగక ఆనర గవస టశక ళనజర

Topic #3:
అన ఆమ వచ అతన నవ టర లన దర అద నట మన నన

Topic #4:
చరణ గమణ చలమయ రమద రమణర షద కరత ళధ భగవత నఏ ఏప ౦ట

Topic #5:
జయచ ణఅబ కనవ టమప రఅబ కధ అఆల ఈఅబ ఆలవ ఈబస నకధ రచద


In [0]:
from pyLDAvis import sklearn as sklearn_lda
import pickle 
import pyLDAvis

In [46]:
prep = sklearn_lda.prepare(lda,core,vect,mds='mmds',sort=True)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [47]:
pyLDAvis.display(prep)